In [1]:
#@title __Install all the Requirements__
!pip install diffusers transformers accelerate scipy safetensors &> /dev/null

!pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git &> /dev/null
!pip install py-real-esrgan &> /dev/null

!pip install pyngrok &> /dev/null

In [ ]:
#@title __Import Libraries and Load Models__
import requests
from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionUpscalePipeline
from diffusers import StableDiffusionInpaintPipeline
from diffusers import AutoPipelineForInpainting
from py_real_esrgan.model import RealESRGAN
import torch

# load model and scheduler
model_upscale = "stabilityai/stable-diffusion-x4-upscaler"
pipeline = StableDiffusionUpscalePipeline.from_pretrained(model_upscale, torch_dtype=torch.float16)
pipeline = pipeline.to("cuda")

# Load the Stable Diffusion model for inpainting
model_inpaint = "stabilityai/stable-diffusion-2-inpainting"
inpaint_pipeline = StableDiffusionInpaintPipeline.from_pretrained(model_inpaint, torch_dtype=torch.float16)
inpaint_pipeline = inpaint_pipeline.to("cuda")

# Load the Real-ESRGAN model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RealESRGAN(device, scale=4)
model.load_weights('weights/RealESRGAN_x4.pth', download=True)

# Load Inpainting AI
ai_inpaint = AutoPipelineForInpainting.from_pretrained("diffusers/stable-diffusion-xl-1.0-inpainting-0.1", torch_dtype=torch.float16, variant="fp16").to("cuda")

In [ ]:
#@title __Setup Ngrok API URL__
#@markdown Provide the _auth_token_ with your ngrok token, run and copy it then put in _.env_ on the project.

from pyngrok import ngrok

# Terminate open tunnel if exist
ngrok.kill()

port = 5000

auth_token = "" #@param {type:"string"}
ngrok.set_auth_token(auth_token)

ngrok.connect(port).public_url

In [ ]:
#@title __Run the Flask App__
#@markdown Make sure to re-run the Ngrok Setup first and update _.env_ if you restart this cell.

from flask import Flask, request, send_file, jsonify
import numpy as np

app = Flask(__name__)

@app.route('/', methods=['GET'])
def home():
    return 'Hello, World!'


@app.route('/upscale', methods=['POST'])
def upscale():
  try:
    # Get the image and prompt from the request
    image_data = request.files['image'].read()
    prompt = request.form['prompt']

    # Load the image from the request
    low_res_img = Image.open(BytesIO(image_data)).convert("RGB")

    # Perform the upscaling
    upscaled_image = pipeline(prompt=prompt, image=low_res_img).images[0]

    # Save the upscaled image to a BytesIO buffer
    buffered = BytesIO()
    upscaled_image.save(buffered, format="PNG")
    buffered.seek(0)

    return send_file(buffered, mimetype='image/png', as_attachment=True, download_name='upsampled_image.png')
  except Exception as e:
    return jsonify({'error': str(e)}), 500


@app.route('/inpaint', methods=['POST'])
def inpaint():
  try:
    # Get the image, mask, and prompt from the request
    image_data = request.files['image'].read()
    mask_data = request.files['mask'].read()
    prompt = request.form['prompt']

    # Load the image and mask from the request
    image = Image.open(BytesIO(image_data)).convert("RGB")
    mask = Image.open(BytesIO(mask_data)).convert("L")  # Convert mask to grayscale

    # Perform the inpainting using the Stable Diffusion model
    inpainted_image = inpaint_pipeline(prompt=prompt, image=image, mask_image=mask).images[0]

    # Save the restored image to a BytesIO buffer
    buffered = BytesIO()
    inpainted_image.save(buffered, format="PNG")
    buffered.seek(0)

    return send_file(buffered, mimetype='image/png', as_attachment=True, download_name='inpainted_image.png')

  except Exception as e:
    return jsonify({'error': str(e)}), 500

@app.route('/enhance', methods=['POST'])
def enhance():
  try:
    # Get the image from the request
    image_data = request.files['image'].read()

    # Load the image from the request
    image = Image.open(BytesIO(image_data)).convert('RGB')

    # Perform the upscaling using Real-ESRGAN
    sr_image = model.predict(image)

    # Save the upscaled image to a BytesIO buffer
    buffered = BytesIO()
    sr_image.save(buffered, format="PNG")
    buffered.seek(0)

    return send_file(buffered, mimetype='image/png', as_attachment=True, download_name='enhanced_image.png')

  except Exception as e:
    return jsonify({'error': str(e)}), 500

@app.route('/restore', methods=['POST'])
def restore():
  try:
    # Get the image, mask, and prompt from the request
    image_data = request.files['image'].read()
    mask_data = request.files['mask'].read()
    prompt = request.form['prompt']
    generator = torch.Generator(device="cuda").manual_seed(0)

    # Load the image and mask from the request
    image = Image.open(BytesIO(image_data)).convert("RGB")
    mask = Image.open(BytesIO(mask_data)).convert("L")  # Convert mask to grayscale

    # Perform the inpainting using the Stable Diffusion model
    restored_image = ai_inpaint(
      prompt=prompt,
      image=image,
      mask_image=mask,
      guidance_scale=8.0,
      num_inference_steps=20,  # steps between 15 and 30 work well for us
      strength=0.99,  # make sure to use `strength` below 1.0
      generator=generator,
    ).images[0]

    # Save the restored image to a BytesIO buffer
    buffered = BytesIO()
    restored_image.save(buffered, format="PNG")
    buffered.seek(0)

    return send_file(buffered, mimetype='image/png', as_attachment=True, download_name='restored_image.png')

  except Exception as e:
    return jsonify({'error': str(e)}), 500


if __name__ == '__main__':
    app.run()